In [4]:
import numpy as np
seed = 42
np.random.seed(seed)

In [5]:
from sklearn.metrics import confusion_matrix
import cv2
import copy
import matplotlib.pyplot as plt
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D,BatchNormalization,GaussianNoise
from tensorflow.keras.layers import MaxPool2D,ZeroPadding2D,MaxPooling2D
from tensorflow.keras.layers import Flatten,Activation
from tensorflow.keras.layers import Dense,Dropout
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import Adam,SGD
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras import initializers
import numpy as np
from keras import regularizers

from sklearn.preprocessing import LabelEncoder
from keras.utils.np_utils import to_categorical
from sklearn.metrics import confusion_matrix,classification_report
from sklearn.metrics import auc,roc_curve

#import tensorflow as tf
#tf.logging.set_verbosity(tf.logging.ERROR)

from sklearn.model_selection import train_test_split
from keras.utils.np_utils import to_categorical # convert to one-hot-encoding
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPool2D
from keras.optimizers import RMSprop
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import ReduceLROnPlateau
from glob import glob
import os

In [6]:
target=112


batchsize =16

#data_path id
data_path=r"C:\Users\NRAD.NRAD-28\Desktop\tumordata\tumor\image\image\id_"

pngs=[]

#pngs=glob(data_path)

for i in range(1,3065):
    pngs.append(data_path+str(i)+'.png')

In [7]:
import pandas as pd

#the csv label id path

df=pd.read_csv(r"C:\Users\NRAD.NRAD-28\Desktop\tumordata\tumor\mat2python_mri_tumor.csv")

y_true=[]

for i in range(3064):
    
    y_true.append(int(df['CLASS NO'][i]))

In [8]:
set(y_true)

{1, 2, 3}

In [9]:
encoder = LabelEncoder()
encoder.fit(y_true)
y_true = encoder.transform(y_true)
y_true = to_categorical(y_true)

In [10]:
#set(y_true)

In [11]:
train_datagen = ImageDataGenerator(rescale=1./255,
                                   rotation_range=20,
                                   width_shift_range=0.2,
                                   height_shift_range=0.2,
                                   horizontal_flip=True,
                                   fill_mode='nearest')

test_datagen = ImageDataGenerator(rescale=1./ 255)

In [12]:
earlystop = EarlyStopping(monitor = 'val_loss', 
                          min_delta = 0, 
                          patience = 2,
                          verbose = 1,
                          restore_best_weights = True)

reduce_lr = ReduceLROnPlateau(monitor = 'val_loss', factor = 0.2, patience = 3, verbose = 1, min_delta = 0.0001)

# we put our call backs into a callback list
callbacks = [earlystop, reduce_lr]

In [27]:
from keras.applications.inception_v3 import InceptionV3
from keras.applications.vgg16 import VGG16
from keras.applications.vgg19 import VGG19
from keras.layers import Input,GlobalAveragePooling2D
from keras.models import Model

In [28]:
base_model = VGG16(weights='imagenet',input_shape = (target, target, 3), include_top=False)

for (i,layer) in enumerate(base_model.layers):
    print(str(i) + " "+ layer.__class__.__name__, layer.trainable)
x = base_model.output
x = Flatten(name = "flatten")(x)# let's add a fully-connected layer
x = Dense(512,kernel_regularizer=regularizers.l2(0.00001),activation='relu')(x)
x = BatchNormalization()(x)
x = Dropout(0.5)(x)

x = Dense(256,kernel_regularizer=regularizers.l2(0.00001),activation='relu')(x)
x = BatchNormalization()(x)
x = Dropout(0.5)(x)

# and a logistic layer -- let's say we have 200 classes
predictions = Dense(3, activation='softmax')(x)

AttributeError: module 'tensorflow' has no attribute 'get_default_graph'

In [23]:
# this is the model we will train
model = Model(inputs=base_model.input, outputs=predictions)

NameError: name 'predictions' is not defined

In [ ]:
from IPython.display import SVG
from keras.utils.vis_utils import model_to_dot

SVG(model_to_dot(model).create(prog='dot', format='svg'))

In [19]:
for layer in base_model.layers:
    layer.trainable = False

In [20]:

for (i,layer) in enumerate(base_model.layers):
    print(str(i) + " "+ layer.__class__.__name__, layer.trainable)

0 InputLayer False
1 Conv2D False
2 Conv2D False
3 MaxPooling2D False
4 Conv2D False
5 Conv2D False
6 MaxPooling2D False
7 Conv2D False
8 Conv2D False
9 Conv2D False
10 MaxPooling2D False
11 Conv2D False
12 Conv2D False
13 Conv2D False
14 MaxPooling2D False
15 Conv2D False
16 Conv2D False
17 Conv2D False
18 MaxPooling2D False


In [21]:
for (i,layer) in enumerate(model.layers):
    
    print(str(i) + " "+ layer.__class__.__name__, layer.trainable)

NameError: name 'model' is not defined

In [ ]:
dataset=list()

for i in range(len(pngs)):
    
    img=cv2.imread(pngs[i])
    #img=cv2.equalizeHist(img)
    img=cv2.resize(img,(target,target))
    dataset.append(img)

dataset=np.array(dataset)

len(dataset)

In [ ]:
X_trainset,X_test ,y_trainset,y_test= train_test_split(dataset,y_true,test_size=0.2, shuffle=True,random_state=seed)

X_trainset = X_trainset.reshape(-1,target,target,3)
X_test = X_test.reshape(-1,target,target,3)


X_trainset.shape,len(y_trainset)

In [ ]:
X_test = X_test/np.max(X_test)

In [ ]:
#X_test=test_datagen.flow(X_test)


In [ ]:
X_test.shape[0]

In [ ]:
X_train,X_valid,y_train,y_valid = train_test_split(X_trainset,y_trainset,test_size=0.3, shuffle=False,random_state=seed)

X_train.shape[0]

In [ ]:
model.compile(optimizer=Adam(0.0001), loss='categorical_crossentropy', metrics=['acc'])

output =model.fit_generator(train_datagen.flow(X_train, y_train, seed=seed,batch_size=batchsize),steps_per_epoch=X_train.shape[0]//batchsize, 
                            epochs=20, verbose=1,callbacks=callbacks,
                            validation_data=test_datagen.flow(X_valid,y_valid,batch_size=batchsize,seed=seed),
                            validation_steps=X_valid.shape[0]//batchsize,shuffle=True)

In [ ]:
plt.plot(output.history['acc'])
plt.plot(output.history['val_acc'])
plt.title('classifier accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()
# summarize history for loss
plt.plot(output.history['loss'])
plt.plot(output.history['val_loss'])
plt.title('classifier loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()

In [ ]:
for (i,layer) in enumerate(model.layers):
    
    print(str(i) + " "+ layer.__class__.__name__, layer.trainable)

In [ ]:
N=15

for layer in model.layers[:N]:
   layer.trainable = False
for layer in model.layers[N:]:
   layer.trainable = True

In [ ]:
for (i,layer) in enumerate(model.layers):
    print(str(i) + " "+ layer.__class__.__name__, layer.trainable)


In [ ]:
model.compile(optimizer=Adam(0.00001), loss='categorical_crossentropy', metrics=['acc'])

output =model.fit_generator(train_datagen.flow(X_train, y_train, batch_size=batchsize),steps_per_epoch=X_train.shape[0]//batchsize, 
                                 epochs=20, verbose=1,callbacks=callbacks,
                                 validation_data=test_datagen.flow(X_valid,y_valid,batch_size=batchsize), validation_steps=X_valid.shape[0]//batchsize,shuffle=True)


In [ ]:
# summarize history for accuracy
plt.plot(output.history['acc'])
plt.plot(output.history['val_acc'])
plt.title('classifier accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()
# summarize history for loss
plt.plot(output.history['loss'])
plt.plot(output.history['val_loss'])
plt.title('classifier loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()


In [ ]:
score=model.evaluate_generator(test_datagen.flow(X_test, y_test, batch_size=batchsize),steps=y_test.shape[0])

In [ ]:
print('test accuracy is: ' + str(score[1]))

print('test loss is: '+ str(score[0]))

In [ ]:
y_test.shape[0]//batchsize

In [ ]:
y_hat = model.predict(X_test, batch_size=2,verbose=1, steps=None)
y_hat_class = y_hat.argmax(axis=-1)

In [ ]:
set(y_hat_class)

In [ ]:
y_test_class = y_test.argmax(axis=-1)

In [ ]:
y_hat_class

In [ ]:
cm=confusion_matrix(y_test_class,y_hat_class)

report=classification_report(y_test_class,y_hat_class)

In [ ]:
def accuracy(confusion_matrix):
    diagonal_sum = confusion_matrix.trace()
    sum_of_all_elements = confusion_matrix.sum()
    return diagonal_sum / sum_of_all_elements 

In [ ]:
accuracy(cm)

In [ ]:
print(cm)
print('\n')

print(report)
print('\n')

In [ ]:
y_hat.shape[0]

In [ ]:
accuracy=(cm[0][0] + cm[1][1] + cm[2][2])*100//y_hat.shape[0]

accuracy

In [ ]:
model.save('best.h5')